In [1]:
import torch
print("cudnn version:{}".format(torch.backends.cudnn.version()))
print("cuda version: {}".format(torch.version.cuda))

cudnn version:None
cuda version: None


In [1]:
import torch
import torchvision.models.video as models
import torch.nn.functional as F

In [2]:
import torch
import torch.nn as nn
class CustomConv3DModel(nn.Module):
    def __init__(self):
        super(CustomConv3DModel, self).__init__()
        self.conv1 = nn.Conv3d(in_channels=3, out_channels=12, kernel_size=(3, 5, 5), padding=(1, 1, 1))
        self.bn1 = nn.BatchNorm3d(12)
        self.pool1 = nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(2, 3, 3))

        self.conv2 = nn.Conv3d(in_channels=12, out_channels=24, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.bn2 = nn.BatchNorm3d(24)
        self.pool2 = nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2))

        self.conv3 = nn.Conv3d(in_channels=24, out_channels=36, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.bn3 = nn.BatchNorm3d(36)
        self.pool3 = nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2))

        self.conv4 = nn.Conv3d(in_channels=36, out_channels=48, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.bn4 = nn.BatchNorm3d(48)
        self.pool4 = nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2))

        self.conv5 = nn.Conv3d(in_channels=48, out_channels=64, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.bn5 = nn.BatchNorm3d(64)
        self.pool5 = nn.MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2))

        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(0.5)

        self.fc1 = nn.Linear(10368, 2048)
        self.fc2 = nn.Linear(2048, 400)
        self.fc3 = nn.Linear(400, 4)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        x = self.pool4(F.relu(self.bn4(self.conv4(x))))
        x = self.pool5(F.relu(self.bn5(self.conv5(x))))
        x = self.flatten(x)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x
model = CustomConv3DModel()
print(model)

CustomConv3DModel(
  (conv1): Conv3d(3, 12, kernel_size=(3, 5, 5), stride=(1, 1, 1), padding=(1, 1, 1))
  (bn1): BatchNorm3d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool3d(kernel_size=(1, 3, 3), stride=(2, 3, 3), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv3d(12, 24, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (bn2): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv3d(24, 36, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (bn3): BatchNorm3d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv3d(36, 48, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (bn4): BatchNorm3d(48, eps=1e-05, momentum=0.1, affine=True, tr

In [3]:
# GPU 사용 가능 여부 확인 후 모델 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

CustomConv3DModel(
  (conv1): Conv3d(3, 12, kernel_size=(3, 5, 5), stride=(1, 1, 1), padding=(1, 1, 1))
  (bn1): BatchNorm3d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool3d(kernel_size=(1, 3, 3), stride=(2, 3, 3), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv3d(12, 24, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (bn2): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv3d(24, 36, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (bn3): BatchNorm3d(36, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv3d(36, 48, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (bn4): BatchNorm3d(48, eps=1e-05, momentum=0.1, affine=True, tr

In [4]:
 torch.cuda.is_available() 

True

In [4]:
import torch
from torch.utils.data import Dataset, Subset, DataLoader, random_split,RandomSampler
from torchvision import transforms
from torchvision.datasets import ImageFolder

class CustomDataset(Dataset):
    def __init__(self, root, batch_size, transform=None):
        self.dataset = ImageFolder(root=root, transform=transform)
        self.batch_size = batch_size

    def __len__(self):
        return len(self.dataset) // self.batch_size

    def __getitem__(self, idx):
        start_idx = idx * self.batch_size
        end_idx = (idx + 1) * self.batch_size
        batch_x = [self.dataset[i][0] for i in range(start_idx, end_idx)]
        batch_y = self.dataset[start_idx][1]  # 배치의 시작 인덱스의 이미지 라벨 사용

        # 이미지 리스트를 텐서로 변환
        batch_x = torch.stack(batch_x)
        # 라벨을 텐서로 변환
        batch_y = torch.tensor(batch_y)
        return batch_x, batch_y

# 데이터셋의 루트 폴더 경로
data_root = "E:/PycharmProjects/peslab/GDSC SC/232.재활용품 분류 및 선별 데이터"

# 데이터 전처리 및 변환 설정
data_transform = transforms.Compose([
    transforms.Resize((500, 500)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 커스텀 데이터셋 인스턴스 생성
frame_size = 3  # 각 배치에 포함될 이미지 개수
custom_dataset = CustomDataset(root=data_root, batch_size=frame_size, transform=data_transform)

# custom_dataset을 무작위로 섞음
# random_sampler = RandomSampler(custom_dataset)
# DataLoader를 사용하여 데이터를 로드하고 분할

batch_size = 3  # 원하는 배치 크기로 설정해서 불러옴
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle = True)
# 데이터셋을 트레이닝셋과 테스트셋으로 나누기
# train_indices = range(0, len(custom_dataset) // 2)
# test_indices = range(len(custom_dataset) // 2, len(custom_dataset))

# train_dataset = Subset(custom_dataset, train_indices)
# test_dataset = Subset(custom_dataset, test_indices)
train_ratio = 0.8
train_size = int(train_ratio * len(custom_dataset))
test_size = len(custom_dataset) - train_size

#아 이게 나뉜다음에 섞이는구나 그래서 저걸 먼저 해야되는구나 RandomSampler
train_dataset, test_dataset = random_split(data_loader.dataset, [train_size, test_size])


# 데이터 로더 생성
# data_loader = torch.utils.data.DataLoader(custom_dataset, batch_size=3, shuffle=True)
# 데이터로더 생성
batch_size = 3
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
import pandas as pd
action_loss_df = pd.DataFrame(columns=['iteration','num', 'loss'])

In [8]:
import torch
import torchvision.models.video as models
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch.nn.functional as F

# 학습 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 5  # 학습 에폭 수 설정
df_num = 1

num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    s = 0
    n = 1
    for inputs, labels in train_loader:
        # inputs = inputs.unsqueeze(0).expand(3, -1, -1, -1, -1)
        inputs, labels = inputs.to(device), labels.to(device)
        print(inputs.shape)
        # inputs = inputs.permute(0, 2, 1, 3, 4)
        optimizer.zero_grad()
        print(labels)
        # print(labels.shape)
        outputs = model(inputs)
        loss = F.cross_entropy(outputs, labels)
        # loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        s += len(train_loader)
        print("Completed : ", n, "The rest :", len(train_loader)-n)
        n += 1

        action_loss_df.loc[df_num]=[epoch+1, n, str(loss.item())]
        df_num += 1
print("Training complete!")

torch.Size([3, 3, 3, 500, 500])
tensor([0, 0, 0], device='cuda:0')
Completed :  1 The rest : 5217
torch.Size([3, 3, 3, 500, 500])
tensor([0, 0, 0], device='cuda:0')
Completed :  2 The rest : 5216
torch.Size([3, 3, 3, 500, 500])
tensor([0, 0, 0], device='cuda:0')
Completed :  3 The rest : 5215
torch.Size([3, 3, 3, 500, 500])
tensor([0, 0, 0], device='cuda:0')
Completed :  4 The rest : 5214
torch.Size([3, 3, 3, 500, 500])
tensor([0, 0, 0], device='cuda:0')
Completed :  5 The rest : 5213
torch.Size([3, 3, 3, 500, 500])
tensor([0, 0, 0], device='cuda:0')
Completed :  6 The rest : 5212
torch.Size([3, 3, 3, 500, 500])
tensor([0, 0, 0], device='cuda:0')
Completed :  7 The rest : 5211
torch.Size([3, 3, 3, 500, 500])
tensor([0, 0, 0], device='cuda:0')
Completed :  8 The rest : 5210
torch.Size([3, 3, 3, 500, 500])
tensor([0, 0, 0], device='cuda:0')
Completed :  9 The rest : 5209
torch.Size([3, 3, 3, 500, 500])
tensor([0, 0, 0], device='cuda:0')
Completed :  10 The rest : 5208
torch.Size([3, 3, 3

In [12]:
action_loss_df.to_csv("E:/PycharmProjects/peslab/GDSC SC/20240123_recycle_loss_df.csv", index=False)

In [13]:
"""import torch
import torchvision.models.video as models
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch.nn.functional as F



# 손실 함수 및 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 루프
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    s = 0
    for inputs, labels in train_loader:
        # inputs = inputs.unsqueeze(0).expand(3, -1, -1, -1, -1)
        inputs, labels = inputs.to(device), labels.to(device)
        print(inputs.shape)
        # inputs = inputs.permute(0, 2, 1, 3, 4)
        optimizer.zero_grad()
        print(labels)
        # print(labels.shape)
        outputs = model(inputs)
        loss = F.cross_entropy(outputs, labels)
        # loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        s += len(train_loader)
        running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {running_loss / s}")
print("Training complete!")"""

'import torch\nimport torchvision.models.video as models\nimport torch.nn as nn\nimport torch.optim as optim\nfrom torchvision import transforms\nfrom torch.utils.data import DataLoader\nfrom torchvision.datasets import ImageFolder\nimport torch.nn.functional as F\n\n\n\n# 손실 함수 및 옵티마이저 정의\ncriterion = nn.CrossEntropyLoss()\noptimizer = optim.Adam(model.parameters(), lr=0.001)\n\n# 학습 루프\nnum_epochs = 5\nfor epoch in range(num_epochs):\n    model.train()\n    running_loss = 0.0\n    s = 0\n    for inputs, labels in train_loader:\n        # inputs = inputs.unsqueeze(0).expand(3, -1, -1, -1, -1)\n        inputs, labels = inputs.to(device), labels.to(device)\n        print(inputs.shape)\n        # inputs = inputs.permute(0, 2, 1, 3, 4)\n        optimizer.zero_grad()\n        print(labels)\n        # print(labels.shape)\n        outputs = model(inputs)\n        loss = F.cross_entropy(outputs, labels)\n        # loss = criterion(outputs, labels)\n        loss.backward()\n        optimizer.s

In [14]:
# 모델의 전체 구조와 가중치를 저장
save_path ='E:/PycharmProjects/peslab/GDSC SC/c3d_recycle_weights_withTwo_20230117.pth'
torch.save(model.state_dict(), save_path)
print("Model has been saved.")

Model has been saved.


#pooling층의 stride 중요하다.
#첫번째 conv layer 커널 2 5 5 로 하는게 의미가 있나보다. 처음의 커널은 크게 잡는다. 그것도 2채널로 앞에
#그다음은 1채널 3 * 3 으로 잡았다.
#stride 하는게 로스와 용량 줄이는데 굉장히 중요

#conv 마다 dropout 해주면 안된다.
#마지막 펼쳐주고 fc 지나기 직전에 dropout 해줘야 성능이 잘 나온다.

# 로스 30에서 0.05로 줄이는거 성공했다.
#RAM 50GB에서 1.8GB로 줄이는데 성공했다.

# 개 쩐 다.

In [15]:
# 모델 평가
def evaluate_model(loader, model):
    model.eval()  # 모델을 평가 모드로 전환
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            print(labels)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total * 100
    return accuracy

# 테스트 데이터로 모델 평가
test_accuracy = evaluate_model(test_loader, model)
print(f"Test Accuracy: {test_accuracy:.2f}%")

tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], device='cuda:0')
tensor([0, 0, 0], de